<a href="https://colab.research.google.com/github/nverchev/other_Python_projects/blob/main/Chamfer_distance_for_Pytorch_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the best implementation of the Chamfer Distance?

We compare a simple implementation with other two coming from two libraries.
- https://github.com/otaheri/chamfer_distance
- https://pypi.org/project/chamferdist/1.0.0/#description

These implementatations are compatible with torch.autograd, and we test them using GPU power.

I suggest to run the experiment different times to account for some possible overhead.

---
# Update
In terms of effiency AND scalability the better option seems to be [KeOps](https://www.kernel-operations.io/keops/index.html).
Unfortunately the min() operation is not supported for backprop.
The provided code uses the index to recalculate the distances in torch.


In [92]:
#First implementation 
!pip install git+'https://github.com/otaheri/chamfer_distance'

#Second implementation
!pip install chamferdist


  Cloning https://github.com/otaheri/chamfer_distance to /tmp/pip-req-build-pli7hdsh
  Running command git clone -q https://github.com/otaheri/chamfer_distance /tmp/pip-req-build-pli7hdsh
     |████████████████████████████████| 108 kB 5.3 MB/s 
  Created wheel for chamfer-distance: filename=chamfer_distance-0.1-py3-none-any.whl size=5653 sha256=84e16ae4449269ca5b8b38921a724bdbaec36a87e87212e1e92b41222c416958
  Stored in directory: /tmp/pip-ephem-wheel-cache-8x5db3zx/wheels/2a/c5/7c/395771526a57f81590f5b9e2be57f219f834d894e10b1cd993
Successfully built chamfer-distance
  Created wheel for chamferdist: filename=chamferdist-1.0.0-cp37-cp37m-linux_x86_64.whl size=5836276 sha256=7817a636b732c8407dbc2a7874a0e19435ebf8f8ee8aa44d2cf80d1fed00346e
  Stored in directory: /root/.cache/pip/wheels/28/bb/d1/c789ecd6835e466e813f6e2c5e23bb1bbb2248e84586ba82d2
Successfully built chamferdist


In [1]:
!pip install pykeops > install.log

In [98]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pc1 = torch.rand([100,1000,3]).to()
pc2 = torch.rand([100,100,3]).to()

In [99]:
from chamfer_distance import ChamferDistance
chamdist = ChamferDistance()
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d1,d2,_,_ = chamdist(pc1,pc2) # dist forward, dist reverse
d = d1.sum(axis=1).mean() + d2.sum(axis=1).mean() # batchmean
end.record()
torch.cuda.synchronize()

print("First implementation:")
print("Time (ms): ", start.elapsed_time(end))
print("Result: ", d)

First implementation:
Time (ms):  199.6451873779297
Result:  tensor(19.5153)


In [100]:
from chamferdist import ChamferDistance
chamdist = ChamferDistance()
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d = chamdist(pc1,pc2, bidirectional=True, reduction="mean") 
end.record()
torch.cuda.synchronize()

print("Second implementation:")
print("Time (ms): ", start.elapsed_time(end))
print("Result: ", d)

Second implementation:
Time (ms):  183.89068603515625
Result:  tensor(19.5153)


In [101]:
def square_distance(t1, t2):
    t2 = t2.permute(0, 2, 1)
    dist = -2 * torch.matmul(t1, t2)
    dist += torch.sum(t1 ** 2, 2, keepdim=True)
    dist += torch.sum(t2 ** 2, 1, keepdim=True)
    return dist
    
# Chamfer Distance
def chamdist(t1, t2): 
    dist = square_distance(t1, t2)
    # forward + reverse
    return torch.min(dist, axis = 2)[0].mean(0).sum()\
         + torch.min(dist, axis = 1)[0].mean(0).sum()

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d = chamdist(pc1, pc2)
end.record()
torch.cuda.synchronize()

print("Simple implementation:")
print("Time: ", start.elapsed_time(end))
print("Result: ", d)

Simple implementation:
Time:  69.26172637939453
Result:  tensor(19.5153)


In [103]:
from pykeops.torch import LazyTensor
def square_distance(t1, t2):
    t1 = LazyTensor(t1[:, :, None, :])
    t2 = LazyTensor(t2[:, None,:, :])
    dist = ((t1 - t2) ** 2).sum(-1)
    return dist
    
# Chamfer Distance
def chamdist(t1, t2): 
    dist = square_distance(t1, t2)
    # The following code is currently not supported for backprop
    # return dist.min(axis = 2).mean(0).sum()\
    #      + dist.min(axis = 1).mean(0).sum()

    # We use the retrieved index on torch
    idx1 = dist.argmin(axis = 1).expand(-1, -1, 3)
    m1 = t1.gather(1, idx1)
    s1 = ((t2 - m1) ** 2).mean(0).sum()
    idx2 = dist.argmin(axis = 2).expand(-1, -1, 3)
    m2 = t2.gather(1, idx2)
    s2 = ((t1 - m2) ** 2).mean(0).sum()
    # forward + reverse
    return s1 + s2

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d = chamdist(pc1, pc2)
end.record()
torch.cuda.synchronize()

print("Simple implementation:")
print("Time: ", start.elapsed_time(end))
print("Result: ", d)

Simple implementation:
Time:  5.831488132476807
Result:  tensor(19.5153)


# Discussion
Surprisingly, the simple implementation in torch is quicker than the ones from the two libraries. KeOps is by far the best backend, even without the backprop support for min().
Hope this could be useful to anybody starting with PCD.
